In [37]:
import numpy as np
import pandas as pd
import vtk
import os

np.random.seed(23765)

In [ ]:
column_headers_pos = ['local id', 'pos x', 'pos y', 'pos z', 'area', 'type']

df_pos = pd.read_csv('rank_0_positions.txt', skiprows= 8, delimiter=' ', names=column_headers_pos)
df_pos.to_csv('positions.csv', index=False)

df_pos.head()

,local id,pos x,pos y,pos z,area,type
0,1,88.017654,143.88911,83.259131,area_43,ex
1,2,88.071787,143.79220,83.237149,area_43,ex
2,3,88.093434,143.95190,83.168331,area_43,ex
3,4,88.063898,143.81981,83.317563,area_43,ex
4,5,88.066131,143.97354,83.238470,area_43,ex


In [41]:
neurons = np.sort(np.random.choice(np.arange(0, 50000, 1), size=500, replace=False))

In [42]:
column_headers_monitors = ['step', 'calcium']

df_calcium = pd.DataFrame()

for i in neurons:
    filepath = f'/Users/Kattelijn/Desktop/viz-no-network/monitors/0_{i}.csv'
    temp_df = pd.read_csv(filepath, skiprows=0, usecols=[0, 5], delimiter=';', names=column_headers_monitors)
    temp_df['neuron_id'] = i  # Add the neuron ID as a new column
    df_calcium = pd.concat([df_calcium, temp_df], ignore_index=True)

df_calcium.tail()

,step,calcium,neuron_id
4999995,999500,0.695479,49988
4999996,999600,0.694538,49988
4999997,999700,0.693605,49988
4999998,999800,0.694667,49988
4999999,999900,0.695713,49988


In [43]:
def export_to_vtp(df, folder, filename):
    # Ensure the folder exists
    if not os.path.exists(folder):
        os.makedirs(folder)

    # Full path to the file
    filepath = os.path.join(folder, filename)

    # Create a VTK points object
    points = vtk.vtkPoints()

    # Add points to the object
    for _, row in df.iterrows():
        points.InsertNextPoint(row['pos x'], row['pos y'], row['pos z'])

    # Create a polydata object
    polydata = vtk.vtkPolyData()
    polydata.SetPoints(points)

    # Add calcium data as a point scalar
    calcium_array = vtk.vtkFloatArray()
    calcium_array.SetName("Calcium Concentration")
    for _, row in df.iterrows():
        calcium_array.InsertNextValue(row['calcium'])

    polydata.GetPointData().SetScalars(calcium_array)

    # Write to a VTP file
    writer = vtk.vtkXMLPolyDataWriter()
    writer.SetFileName(filepath)
    writer.SetInputData(polydata)

    # Ensure file is properly written
    writer.SetDataModeToAscii()  # Optional: Change to Binary for larger datasets
    writer.Write()

In [44]:
folder = "500neurons_vtp_files"
time_steps = np.arange(0, 100000, 1000)

for time_point in time_steps:
    # Filter calcium data for the current time step
    df_calcium_at_time = df_calcium[df_calcium['step'] == time_point]

    # Merge with positions
    df_merged = pd.merge(df_pos, df_calcium_at_time, left_on='local id', right_on='neuron_id')

    # Export to VTP file
    filename = f'neurons_timestep_{time_point}.vtp'
    export_to_vtp(df_merged, folder, filename)

    print(f"Saved: {os.path.join(folder, filename)}")

Saved: 500neurons_vtp_files/neurons_timestep_0.vtp
Saved: 500neurons_vtp_files/neurons_timestep_1000.vtp
Saved: 500neurons_vtp_files/neurons_timestep_2000.vtp
Saved: 500neurons_vtp_files/neurons_timestep_3000.vtp
Saved: 500neurons_vtp_files/neurons_timestep_4000.vtp
Saved: 500neurons_vtp_files/neurons_timestep_5000.vtp
Saved: 500neurons_vtp_files/neurons_timestep_6000.vtp
Saved: 500neurons_vtp_files/neurons_timestep_7000.vtp
Saved: 500neurons_vtp_files/neurons_timestep_8000.vtp
Saved: 500neurons_vtp_files/neurons_timestep_9000.vtp
Saved: 500neurons_vtp_files/neurons_timestep_10000.vtp
Saved: 500neurons_vtp_files/neurons_timestep_11000.vtp
Saved: 500neurons_vtp_files/neurons_timestep_12000.vtp
Saved: 500neurons_vtp_files/neurons_timestep_13000.vtp
Saved: 500neurons_vtp_files/neurons_timestep_14000.vtp
Saved: 500neurons_vtp_files/neurons_timestep_15000.vtp
Saved: 500neurons_vtp_files/neurons_timestep_16000.vtp
Saved: 500neurons_vtp_files/neurons_timestep_17000.vtp
Saved: 500neurons_vtp_f